In [2]:
import sys
from pathlib import Path

sys.path.append("..")
from ribasim_lumping import RibasimLumpingNetwork

import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr
import dfm_tools as dfmt
import xugrid as xu
import matplotlib.pyplot as plt

from numba.core.errors import NumbaDeprecationWarning, NumbaPendingDeprecationWarning
import warnings

warnings.simplefilter("ignore", category=NumbaDeprecationWarning)
warnings.simplefilter("ignore", UserWarning)

import networkx as nx

In [3]:
%load_ext autoreload
%autoreload 2

##### STEP 1 - READ NETWORK AND SIMULATION DATA
Read network and simulation data from D-Hydro simulations using his.nc and map.nc:
- for sets WINTER and SUMMER and ...
- for list of SIMULATIONS per set (in this case 1 simulation per set)
- at list of TIMESTEPS per simulation (in this case multiple timesteps per simulation)

and read network

In [4]:
# Define base_dir, results_dir and network name
base_dir = Path("..\\..\\ribasim_lumping_data\\")
dhydro_dir = Path(base_dir, "d-hydro")
results_dir = Path(base_dir, "results")

In [5]:
network_name = "tki_zutphen_berkel"

simulation_name = 'tki_zuthpen_berkel_basis.dsproj_data' # nog verwerken in de code
file_bc = f'{base_dir}\\d-hydro\\{simulation_name}\\FlowFM\\input\\FlowFM_boundaryconditions1d.bc' # nog verwerken in de code

In [6]:
# Load areas (discharge units: afwaterende eenheden)
areas_file_path = Path(base_dir, "afw_eenheden", f"{network_name}_afw_eenheden.shp")
areas_gdf = gpd.read_file(areas_file_path)
areas_gdf = areas_gdf[['GFEIDENT', 'geometry']]

In [7]:
# Create networkanalysis
network = RibasimLumpingNetwork(
    name=network_name, 
    results_dir=results_dir,
    areas_gdf=areas_gdf,
)
# network.export_to_geopackage()

In [8]:
# Select simulation sets and extract all data using xugrid/ugrid
network.add_data_from_simulations_set(
    set_name="winter",
    simulations_dir=dhydro_dir,
    simulations_names=["tki_zuthpen_berkel_basis.dsproj_data"],
    simulation_output_dir="FlowFM\\output",
    simulations_ts=pd.date_range("2000-01-02 23:00", periods=9, freq="2D"),
);

 - Simulation set (winter): tki_zuthpen_berkel_basis.dsproj_data | Timestamps: 9 | his.nc and map.nc


In [9]:
# Read network data and extract all objects (weirs/pumps/laterals/confluences/bifurcations)
network.get_network_data(file_bc)
network.export_to_geopackage()

D-HYDRO-network analysed:
 - nodes and waterlevels
 - edges and discharges
 - stations / pumps / weirs / orifices / bridges / culverts / uniweirs / confluences / bifurcations / boundaries
Exporting to geopackage:
 - areas, nodes, edges, stations, pumps, weirs, orifices, bridges, culverts, uniweirs, boundaries, 
 - not available: basin_areas, split_nodes, basins, basin_connections, boundary_basin_connections, 
Export location: ..\..\ribasim_lumping_data\results\tki_zutphen_berkel\ribasim_network.qgz


##### STEP 2: SPLIT NETWORK IN RIBASIM BASINS
2. Split network in ribasim basins  
    1. Specify model cut locations (id nodes or xy coordinates)  
    2. Dividing the network based on the clipping locations (function to be built in xugrid?)  
        1. Convert to networkx graph
        2. Split network
        3. TODO: What to do if you want to distinguish areas that drain into a main watercourse as a basin with the end of the inflow point? Have choices made between:</dd>
            - simple: also cut the main waterway at this junction or
            - difficult: continue to see the parts upstream and downstream of the inflow point together as one basin
    3. Based on the split network, associated drain areas are determined by means of spatial join.
    4. Use representative locations of areas as node location.

In [10]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(weirs=True);

48 split locations
 - weir: 48


In [11]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

Create basins using split nodes:
 - create network graph from nodes/edges
 - split network graph at split locations
 - define numbers Ribasim-Basins and join edges/nodes
 - check whether each split location results in a split
 - create final locations Ribasim-Basins
 - define for each Ribasim-Basin the related basin area
 - create Ribasim-Edges between Boundary and Basin
 - create Ribasim-Edges between Basins and split locations
Exporting to geopackage:
 - areas, nodes, edges, stations, pumps, weirs, orifices, bridges, culverts, uniweirs, basin_areas, split_nodes, basins, basin_connections, boundaries, boundary_basin_connections, 
 - not available: 
Export location: ..\..\ribasim_lumping_data\results\tki_zutphen_berkel\ribasim_network.qgz


In [12]:
# Define locations where the network should be split into Ribasim basins:
network.add_split_nodes(
    weirs=True,
    pumps=True,
    uniweirs=True,
    structures_ids_to_include=[
        'kdu_DR80760025', # duiker vispassage bovenstrooms
        'kst_ST80830001', 'kst_ST80810015', # onderdoorlaten bij verdeelpunt De Berkel (Zutphen)
        'kdu_DR84930010', # duiker met terugslagklep Zutphen Noorderhaven (parallel aan gemaal)
        'kdu_DR80950033', # duikers voor wijk Leesten
        'kdu_DR80940046', 'kdu_DR80950043', 'kdu_DR80950151', # duikers voor wijk Zuidwijken
        'kdu_DR80950103',
    ], 
    structures_ids_to_exclude=[
        'BCAL_3', 'BCAL_11', # stuwen voor hoogwaterafvoer De Berkel
        'BBypass_Besselink_1', 'BBypass_Besselink_2', 'BBypass_Besselink_3', 'BBypass_Besselink_4', 'BBypass_Besselink_5', # visdrempels vispassage De Berkel
        'kst_ST80950035', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Leesten)
        'kst_ST84930001', # verwarrende stuw ivm afwaterende eenheid (Zutphen: Noorderhaven)
    ], 
    node_ids_to_include=[
        419, # voorbeeld splitsing
        # 1455, # extra punt rondom verdeelpunt De Berkel
    ],
    node_ids_to_exclude=[],
);

69 split locations
 - pump: 4
 - weir: 46
 - uniweir: 9
 - orifice: 2
 - culvert: 7
 - node: 1


In [13]:
# Create basins (gdf) based on nodes, edges, split_node_ids and areas
network.create_basins_and_connections_based_on_split_nodes();
# Export to geopackage
network.export_to_geopackage()

Create basins using split nodes:
 - create network graph from nodes/edges
 - split network graph at split locations
 - define numbers Ribasim-Basins and join edges/nodes
 - check whether each split location results in a split
 - create final locations Ribasim-Basins
 - define for each Ribasim-Basin the related basin area
 - create Ribasim-Edges between Boundary and Basin
 - create Ribasim-Edges between Basins and split locations
Exporting to geopackage:
 - areas, nodes, edges, stations, pumps, weirs, orifices, bridges, culverts, uniweirs, basin_areas, split_nodes, basins, basin_connections, boundaries, boundary_basin_connections, 
 - not available: 
Export location: ..\..\ribasim_lumping_data\results\tki_zutphen_berkel\ribasim_network.qgz


##### STEP 3: CREATE SETTINGS RIBASIM BASINS
3. Create settings ribasim basins
    1. Determine volume-water level relations for the Basins. Relationship between total volume and water levels within basin and water level representative outflow point:
        1. Include all disposal in area: storage in calculation nodes and possible disposal nodes.
        2. Also include storage in tertiary system? Modflow or possibly RR?
        3. Practical point we ran into: What to do with the storage in the locations where you are just making the cut? Distribute evenly over the connected basins?
    2. Lump and add lateral inflow/outflow within areas including precipitation/evaporation.
        1. MODFLOW/MetaSWAP: Lump fluxes over area.
        2. D-Hydro: Lump inflow of laterals.


In [14]:
# TODO: CHECK WITH HKV WHAT THEY ARE DOING AND MAYBE IMPORT THEIR FUNCTIONS

In [16]:
ribasim_model = network.generate_ribasim_model()

In [18]:
# ribasim_model.write(f"{results_dir}/{network.name}")

In [20]:
network.split_nodes

,mesh1d_node_id,mesh1d_nEdges,geometry,object_type,projection_x,projection_y,mesh1d_nNodes,basin,status,split_type
0,kgm_GM80920002,104,POINT (216581.528 457573.019),pump,216581.527662,457573.019080,-1,NaN,True,pump
1,kgm_GM84930001,1685,POINT (211236.797 463775.030),pump,211236.796738,463775.029554,-1,NaN,True,pump
2,kgm_GM80950124,1691,POINT (210161.557 460774.353),pump,210161.557283,460774.353015,-1,NaN,True,pump
3,kgm_GM80750028,2388,POINT (222853.698 464144.803),pump,222853.697638,464144.802696,-1,NaN,True,pump
4,kst_ST80940035,38,POINT (213450.820 457384.744),weir,213450.819717,457384.743836,-1,NaN,True,weir
...,...,...,...,...,...,...,...,...,...,...
64,kdu_DR80950033,618,POINT (212180.726 459822.375),culvert,212180.725618,459822.375267,-1,NaN,True,culvert
65,kdu_DR80950103,944,POINT (210729.841 460443.188),culvert,210729.840700,460443.187549,-1,NaN,True,culvert
66,kdu_DR84930010,1688,POINT (211235.124 463775.333),culvert,211235.124394,463775.333020,-1,NaN,True,culvert
67,kdu_DR80760025,2015,POINT (219551.087 463577.531),culvert,219551.086748,463577.531439,-1,NaN,True,culvert


##### STEP 4: BOUNDARIES AND BOUNDARY CONDITIONS FOR INFLOW/OUTFLOW POINTS
4. Boundaries and boundary conditions of inflow/outflow points of the system/network
     1. Automatically identify network edges based on network and areas.
     2. Selecting borders and border type

##### STEP 5: CONNECTIONS BETWEEN RIBASIM - BASINS:
5. Connections Between Ribasim-Basins:
    1. Automatically generate connections between Basins. Determine per simulation set the flow and flow direction (winter/summer/supply):
        1. Specify preferred exchange type (maybe per split location or basin)
        2. Determine presence and direction based on flow direction.
        3. Place connector nodes near the cut locations (but not all on top of each other).
    2. Definition of exchange relations based on preference and manual choice model

##### STEP 6: GENERATE SETTINGS FOR EACH CONNECTION:
6. Generate settings for the exchanges:
    1. TabulatedRatingCurve: QH/QV relationships
    2. Manning: Manning and distance
    3. FractionalFlows for splits with distributions
    4. PID controller: Translate control from D-Hydro-RTC

In [ ]:
# TODO: CHECK WITH HKV WHAT THEY ARE DOING AND MAYBE IMPORT THEIR FUNCTIONS

##### STEP 7: ADD ALL TIME-DEPENDENT INPUT
7. Add all time-dependent inputs:
    1. Hydraulic boundary conditions.
    2. Precipitation on and evaporation from open water
    3. Lateral inflow/outflow to groundwater or tertiary system
    4. Surface water abstractions

In [ ]:
# TODO: CHECK WITH HKV WHAT THEY ARE DOING AND MAYBE IMPORT THEIR FUNCTIONS